## 1: Import libraries and dataset
### 1.1: Importing libraries

In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1.2: Importing the dataset 

In [183]:
df = pd.read_csv('reuters_data.csv', encoding='ISO-8859-1')

In [184]:
df

,Unnamed: 0,raw_header,reuters_url,stock,article_publish_date,full_article,processed_header,neg,neu,pos,compound
0,0,BRIEF-Agilent Technologies Q2 GAAP Earnings Pe...,https://www.reuters.com/article/brief-agilent-...,A,2020-05-21 20:26:54+00:00,May 21 (Reuters) - Agilent Technologies Inc:\n...,Agilent Technologies Q2 GAAP Earnings Per Shar...,0.000,0.761,0.239,0.2960
1,1,BRIEF-Agilent PD-L1 Assay Receives FDA Approva...,https://www.reuters.com/article/brief-agilent-...,A,2020-05-18 12:04:40+00:00,May 18 (Reuters) - Agilent Technologies Inc:\n...,Agilent PD-L1 Assay Receives FDA Approval For ...,0.000,0.763,0.237,0.4767
2,2,BRIEF-Agilent Gets FDA Approval For Pd-L1 Comp...,https://www.reuters.com/article/brief-agilent-...,A,2020-04-23 12:18:06+00:00,April 23 (Reuters) - Agilent Technologies Inc:...,Agilent Gets FDA Approval For Pd-L1 Companion ...,0.000,0.763,0.237,0.4767
3,3,BRIEF-Agilent Technologies Enters Into Amendme...,https://www.reuters.com/article/brief-agilent-...,A,2020-04-20 20:53:23+00:00,April 20 (Reuters) - Agilent Technologies Inc:...,Agilent Technologies Enters Into Amendment No....,0.000,0.580,0.420,0.7003
4,4,"BRIEF-Agilent withdraws Q2, FY guidance, cites...",https://www.reuters.com/article/brief-agilent-...,A,2020-04-14 12:04:44+00:00,April 14 (Reuters) - Agilent Technologies Inc:...,"Agilent withdraws Q2, FY guidance, cites coron...",0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
108857,110111,BRIEF-Zumiez Inc September sales rose 12.6 per...,https://www.reuters.com/article/brief-zumiez-i...,ZUMZ,2017-10-04 20:03:04+00:00,Oct 4 (Reuters) - Zumiez Inc\n\n* Zumiez Inc. ...,Zumiez Inc September sales rose 12.6 percent t...,0.000,1.000,0.000,0.0000
108858,110112,BRIEF-Zumiez Inc Q2 loss per share $0.02,https://www.reuters.com/article/brief-zumiez-i...,ZUMZ,2017-09-07 20:12:25+00:00,Sept 8 (Reuters) - Zumiez Inc\n\n* Zumiez Inc ...,Zumiez Inc Q2 loss per share $0.02,0.242,0.526,0.232,-0.0258
108859,110113,BRIEF-Zumiez net sales increased 8.9 pct to $6...,https://www.reuters.com/article/brief-zumiez-n...,ZUMZ,2017-08-02 20:04:51+00:00,Aug 2 (Reuters) - Zumiez Inc\n\n* Zumiez Inc. ...,Zumiez net sales increased 8.9 pct to $67.5 mi...,0.000,0.792,0.208,0.2732
108860,110114,BRIEF-Man Group reports 5.63 pct passive stake...,https://www.reuters.com/article/brief-man-grou...,ZUMZ,2017-07-31 20:57:50+00:00,July 31 (Reuters) - Zumiez Inc-\n\n* Man Group...,Man Group reports 5.63 pct passive stake in Zu...,0.000,0.816,0.184,0.2023


In [185]:
df.head()

,Unnamed: 0,raw_header,reuters_url,stock,article_publish_date,full_article,processed_header,neg,neu,pos,compound
0,0,BRIEF-Agilent Technologies Q2 GAAP Earnings Pe...,https://www.reuters.com/article/brief-agilent-...,A,2020-05-21 20:26:54+00:00,May 21 (Reuters) - Agilent Technologies Inc:\n...,Agilent Technologies Q2 GAAP Earnings Per Shar...,0.0,0.761,0.239,0.2960
1,1,BRIEF-Agilent PD-L1 Assay Receives FDA Approva...,https://www.reuters.com/article/brief-agilent-...,A,2020-05-18 12:04:40+00:00,May 18 (Reuters) - Agilent Technologies Inc:\n...,Agilent PD-L1 Assay Receives FDA Approval For ...,0.0,0.763,0.237,0.4767
2,2,BRIEF-Agilent Gets FDA Approval For Pd-L1 Comp...,https://www.reuters.com/article/brief-agilent-...,A,2020-04-23 12:18:06+00:00,April 23 (Reuters) - Agilent Technologies Inc:...,Agilent Gets FDA Approval For Pd-L1 Companion ...,0.0,0.763,0.237,0.4767
3,3,BRIEF-Agilent Technologies Enters Into Amendme...,https://www.reuters.com/article/brief-agilent-...,A,2020-04-20 20:53:23+00:00,April 20 (Reuters) - Agilent Technologies Inc:...,Agilent Technologies Enters Into Amendment No....,0.0,0.580,0.420,0.7003
4,4,"BRIEF-Agilent withdraws Q2, FY guidance, cites...",https://www.reuters.com/article/brief-agilent-...,A,2020-04-14 12:04:44+00:00,April 14 (Reuters) - Agilent Technologies Inc:...,"Agilent withdraws Q2, FY guidance, cites coron...",0.0,1.000,0.000,0.0000


### 1.3: Check if the data consist of any null values

In [226]:
df.isnull().any()

Unnamed: 0              False
raw_header              False
reuters_url             False
stock                   False
article_publish_date    False
full_article            False
processed_header        False
neg                     False
neu                     False
pos                     False
compound                False
dtype: bool

## 2: Data preprocessing 

### 2.1: We will create a dataset with only 'processed_header' and 'compound' feature

In [221]:
data = df.loc[:,['processed_header','compound']]
data

,processed_header,compound
0,Agilent Technologies Q2 GAAP Earnings Per Shar...,0.2960
1,Agilent PD-L1 Assay Receives FDA Approval For ...,0.4767
2,Agilent Gets FDA Approval For Pd-L1 Companion ...,0.4767
3,Agilent Technologies Enters Into Amendment No....,0.7003
4,"Agilent withdraws Q2, FY guidance, cites coron...",0.0000
...,...,...
108857,Zumiez Inc September sales rose 12.6 percent t...,0.0000
108858,Zumiez Inc Q2 loss per share $0.02,-0.0258
108859,Zumiez net sales increased 8.9 pct to $67.5 mi...,0.2732
108860,Man Group reports 5.63 pct passive stake in Zu...,0.2023


### 2.2: Renaming the column headers with new names

In [222]:
new_header=['headline', 'confidence']
data.columns = new_header
data

,headline,confidence
0,Agilent Technologies Q2 GAAP Earnings Per Shar...,0.2960
1,Agilent PD-L1 Assay Receives FDA Approval For ...,0.4767
2,Agilent Gets FDA Approval For Pd-L1 Companion ...,0.4767
3,Agilent Technologies Enters Into Amendment No....,0.7003
4,"Agilent withdraws Q2, FY guidance, cites coron...",0.0000
...,...,...
108857,Zumiez Inc September sales rose 12.6 percent t...,0.0000
108858,Zumiez Inc Q2 loss per share $0.02,-0.0258
108859,Zumiez net sales increased 8.9 pct to $67.5 mi...,0.2732
108860,Man Group reports 5.63 pct passive stake in Zu...,0.2023


### Note: Create a function that will classify the stock into 3 classes based on confidence scores

In [190]:
def classify(x):
    if(x > 0.37):
        return 1
    elif(x < 0.37 and x > -0.30):
        return 0
    else:
        return -1

### 2.3: Applying the classifiy on each element of 'class' column through lambda function

In [223]:
data['class'] = data['confidence'].apply(lambda x: classify(x))
data

,headline,confidence,class
0,Agilent Technologies Q2 GAAP Earnings Per Shar...,0.2960,0
1,Agilent PD-L1 Assay Receives FDA Approval For ...,0.4767,1
2,Agilent Gets FDA Approval For Pd-L1 Companion ...,0.4767,1
3,Agilent Technologies Enters Into Amendment No....,0.7003,1
4,"Agilent withdraws Q2, FY guidance, cites coron...",0.0000,0
...,...,...,...
108857,Zumiez Inc September sales rose 12.6 percent t...,0.0000,0
108858,Zumiez Inc Q2 loss per share $0.02,-0.0258,0
108859,Zumiez net sales increased 8.9 pct to $67.5 mi...,0.2732,0
108860,Man Group reports 5.63 pct passive stake in Zu...,0.2023,0


### Note: We will keep only the headline and class columns 

In [224]:
data = data.loc[:,['headline','class']]
data

,headline,class
0,Agilent Technologies Q2 GAAP Earnings Per Shar...,0
1,Agilent PD-L1 Assay Receives FDA Approval For ...,1
2,Agilent Gets FDA Approval For Pd-L1 Companion ...,1
3,Agilent Technologies Enters Into Amendment No....,1
4,"Agilent withdraws Q2, FY guidance, cites coron...",0
...,...,...
108857,Zumiez Inc September sales rose 12.6 percent t...,0
108858,Zumiez Inc Q2 loss per share $0.02,0
108859,Zumiez net sales increased 8.9 pct to $67.5 mi...,0
108860,Man Group reports 5.63 pct passive stake in Zu...,0


### 2.4: We will perform preprocessing of the data by 
#### a. Removing all symbols except a-z and A-Z
#### b. Converting the data to lower case
#### c. Replacing trailing spaces with single space

In [193]:
data.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
data['headline'] = data['headline'].str.lower()
data['headline'] = data['headline'].replace('\s+', ' ', regex=True)
data

,headline,class
0,agilent technologies q gaap earnings per share,0
1,agilent pd l assay receives fda approval for u...,1
2,agilent gets fda approval for pd l companion d...,1
3,agilent technologies enters into amendment no ...,1
4,agilent withdraws q fy guidance cites coronavi...,0
...,...,...
108857,zumiez inc september sales rose percent to mln,0
108858,zumiez inc q loss per share,0
108859,zumiez net sales increased pct to million,0
108860,man group reports pct passive stake in zumiez,0


### 3: Creating Train and Test data 

### 3.1: Creating the dependent feature, y that represents if the stock moved up, down or did not move decisvely in any direction

In [194]:
y=data['class'].values
y

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

### 3.2: Creating the independent feature, X where each training example is a stock related news headline/ headlines

In [195]:
X=data['headline'].values
X

array(['agilent technologies q gaap earnings per share ',
       'agilent pd l assay receives fda approval for use as companion diagnostic',
       'agilent gets fda approval for pd l companion diagnostic on dako omnis',
       ..., 'zumiez net sales increased pct to million',
       'man group reports pct passive stake in zumiez',
       'zumiez june same store sales rises percent'], dtype=object)

### 3.3: Splitting the X and y data into Train and Test data

In [196]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [197]:
X_train

array(['embraer counts on brazil s public sector funds to approve boeing jv sources',
       'dominion energy project over the historic james river wins reprieve from d c circuit',
       'walgreens revenue hit by weakness in personal care products otc drugs',
       ...,
       'google facebook execs meet uk govt ministers over no deal bexit telegraph',
       'fiat chrysler s lower cash forecast overshadows special dividend pledge',
       'chip gear makers take a beating after micron s dour forecast'],
      dtype=object)

In [198]:
y_train

array([ 0,  1,  0, ...,  0,  0, -1], dtype=int64)

In [199]:
X_test

array(['lisala to sell million of echo investment shares',
       'bat shares light up as it streamlines new age brands',
       's p nasdaq open at record highs on china data initial trade deal',
       ...,
       'brazil s eletrobras stake could be worth billion source says',
       'ebrd buys slice of national bank of greece covered bond',
       'comcast and national public radio to launch npr one app on xfinity x '],
      dtype=object)

In [200]:
y_test

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## 4: Training our model using the train data derived after splitting
### 4.1: Import the module for vectorizer and training algorithm

In [201]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

### 4.2: Calling the count vectorizer and fitting on train data

In [202]:
# Implementing count vectorizer
# vec = CountVectorizer(ngram_range=(2,2))
vec = CountVectorizer()
X_train = vec.fit_transform(X_train)

### 4.3: Calling the training algorithm and fitting on train data

In [203]:
#classifier=RandomForestClassifier(n_estimators=200, criterion='entropy')
classifier=DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

### 4.4: Vectorizing the test data and predicting the class based on trained model

In [204]:
X_test = vec.transform(X_test)
y_pred = classifier.predict(X_test)

## 5: Analyzing the results
### 5.1: Import and run sklearn metrics libraries to analyze results

In [205]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [208]:
cm = confusion_matrix(y_pred, y_test)
a=accuracy_score(y_pred, y_test)
print(a*100)
print(cm)

95.2448023515723
[[ 3062   313    15]
 [  287 22150   426]
 [   39   473  5894]]


#### Results: We are achieving an accuracy of 95% on our test set which is really good

### 5.2: Predicting the class for training example

In [218]:
import re
headline = "quarter 2 revenues decreases by 0.25%, go buy it. dont buy this one, big losses expected "
head = re.sub(r'[^a-zA-Z]', ' ', headline)
head = head.lower()
head = re.sub(' +', ' ', head)
print(head)
data = [head]
print(data)
vect = vec.transform(data).toarray()
print(vect)
pred = classifier.predict(vect)
print(pred)

quarter revenues decreases by go buy it dont buy this one big losses expected 
['quarter revenues decreases by go buy it dont buy this one big losses expected ']
[[0 0 0 ... 0 0 0]]
[-1]


In [219]:
if pred == 1:
    print("increasing movement")
if pred == 0:
    print("no decisive movement")
if pred == -1:
    print("decreasing movement")

decreasing movement


#### Result: The prediction made for the single training example looks logical as this news headline would likely affect the stock sentiment adversely 

### 5.3: Saving our Model and Vectorizer in a .pkl file

In [209]:
pickle.dump(classifier, open('classifier.pkl','wb'))
pickle.dump(vec, open('vec.pkl','wb'))